In [ ]:
from warnings import filterwarnings

filterwarnings("ignore")

In [ ]:
import datetime
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
import typing as tp
import numpy as np
import pandas as pd


X_type = tp.NewType("X_type", np.ndarray)
X_row_type = tp.NewType("X_row_type", np.ndarray)
Y_type = tp.NewType("Y_type", np.array)
TS_type = tp.NewType("TS_type", pd.Series)
Model_type = tp.TypeVar("Model_type")


def read_timeseries(path_to_df: str = "train.csv") -> TS_type:
    """Функция для чтения данных и получения обучающей и тестовой выборок"""
    df = pd.read_csv(path_to_df)
    df = df[(df['store'] == 1) & (df['item'] == 1)]
    df["date"] = pd.to_datetime(df["date"])
    df = df.set_index("date")
    ts = df["sales"]
    train_ts = ts[:-365]
    test_ts = ts[-365:]
    return train_ts, test_ts


def my_f(
    timeseries,
    model_idx,
    window_size = 7
    ):
    row = {}
    date = timeseries[-feature_window:].index[0]
    row["dayofweek"] = date.dayofweek
    row["quarter"] = date.quarter
    row["month"] = date.month
    row["year"] = date.year
    row["dayofyear"] = date.dayofyear
    row["dayofmonth"] = date.day
    row["weekofyear"] = date.weekofyear
    feature_window = window_size + model_idx
    return 


def extract_hybrid_strategy_features(
    timeseries: TS_type,
    model_idx: int,
    window_size: int = 7
) -> X_row_type:
    """
    Функция для получения вектора фичей согласно гибридной схеме. На вход подаётся временной ряд
    до момента T, функция выделяет из него фичи, необходимые модели под номером model_idx для
    прогноза на момент времени T
    
    Args:
        timeseries --- временной ряд до момента времени T (не включительно), pd.Series с датой 
                       в качестве индекса
        model_idx --- индекс модели, то есть номер шага прогноза, 
                      для которого нужно получить признаки, нумерация с нуля
        window_size --- количество последних значений ряда, используемых для прогноза 
                        (без учёта количества прогнозов с предыдущих этапов)

    Returns:
        Одномерный вектор фичей для модели с индексом model_idx (np.array), 
        чтобы сделать прогноз для момента времени T
    """
    feature_window = window_size + model_idx
    return timeseries[-feature_window:].values


def build_datasets(
    timeseries: TS_type,
    extract_features: tp.Callable[..., X_row_type],
    window_size: int,
    model_count: int
) -> tp.List[tp.Tuple[X_type, Y_type]]:
    """
    Функция для получения обучающих датасетов согласно гибридной схеме
    
    Args:
        timeseries --- временной ряд
        extract_features --- функция для генерации вектора фичей
        window_size --- количество последних значений ряда, используемых для прогноза
        model_count --- количество моделей, используемых для получения предскзаний 

    Returns:
        Список из model_count датасетов, i-й датасет используется для обучения i-й модели 
        и представляет собой пару из двумерного массива фичей и одномерного массива таргетов
    """
    datasets = []
    
    for model_idx in range(model_count):
        feature_window = window_size + model_idx
        features = pd.DataFrame(
            [extract_features(
                timeseries[:date], model_idx, window_size=window_size
            ) for date in timeseries.index[feature_window-1:-1]]
        ).to_numpy()
        
        y = timeseries[feature_window:].values
        
        datasets.append((features, y))         
    
    
    assert len(datasets) == model_count
    return datasets


def predict(
    timeseries: TS_type,
    models: tp.List[Model_type],
    extract_features: tp.Callable[..., X_row_type] = extract_hybrid_strategy_features
) -> TS_type:
    """
    Функция для получения прогноза len(models) следующих значений временного ряда
    
    Args:
        timeseries --- временной ряд, по которому необходимо сделать прогноз на следующие даты
        models --- список обученных моделей, i-я модель используется для получения i-го прогноза
        extract_features --- функция для генерации вектора фичей. Если вы реализуете свою функцию 
                             извлечения фичей для конечной модели, передавайте этим аргументом.
                             Внутри функции predict функцию extract_features нужно вызывать только
                             с аргументами timeseries и model_idx, остальные должны быть со значениями
                             по умолчанию

    Returns:
        Прогноз len(models) следующих значений временного ряда
    """
    forecasted_y = []
    timeseries_ = timeseries
    
    for idx, model in enumerate(models):
        features = extract_features(timeseries_, idx)
        value = model.predict(np.array(features)[None])
        forecasted_y.append(value[0])
        timeseries_.at[timeseries_.index[-1] + datetime.timedelta(days=1)] = value[0]
    
    forecasts = pd.Series(forecasted_y)
    forecasts.index = pd.date_range(start=timeseries.index[-1] + datetime.timedelta(days=1), periods=len(models))
    
    return forecasts
    
    


def train_models(
    train_timeseries: TS_type,
    model_count: int
) -> tp.List[Model_type]:
    """
    Функция для получения обученных моделей
    
    Args:
        train_timeseries --- обучающий временной ряд
        model_count --- количество моделей для обучения согласно гибридной схеме.
                        Прогнозирование должно выполняться на model_count дней вперёд

    Returns:
        Список из len(datasets) обученных моделей
    """
    models = [GradientBoostingRegressor() for _ in range(model_count)]
    datasets = build_datasets(train_timeseries, extract_hybrid_strategy_features, window_size=7, model_count=model_count)
    
    for model, (X, y) in zip(models, datasets):
        model.fit(X, y)
   
    
    assert len(models) == len(datasets)
    return models


def score_models(
    train_ts: TS_type,
    test_ts: TS_type, 
    models: tp.List[Model_type],
    predict: tp.Callable[[TS_type, tp.List[Model_type]], TS_type] = predict
):
    """
    Функция для оценки качества обученных моделей по метрике MSE
    
    Args:
        train_ts --- обучающий временной ряд
        test_ts --- тестовый временной ряд
        models --- список обученных моделей
        predict --- функция для получения прогноза временного ряда

    Returns:
        Усредненное MSE для прогноза моделей по всей тестовой выборке
    """
    predict_len = len(models)
    predictions = []
    targets = []

    for i in range(len(test_ts) - predict_len + 1):
        predictions.extend(list(predict(train_ts, models)))
        targets.extend(list(test_ts[i:i+predict_len]))
        train_ts = train_ts.append(test_ts[i:i+1])

    return sklearn.metrics.mean_squared_error(targets, predictions)

In [ ]:
train, test = read_timeseries('train.csv')

In [ ]:
def create_date_features(date):
    """Создает фичи из даты"""

    row = {}
    row["dayofweek"] = date.dayofweek
    row["quarter"] = date.quarter
    row["month"] = date.month
    row["year"] = date.year
    row["dayofyear"] = date.dayofyear
    row["dayofmonth"] = date.day
    row["weekofyear"] = date.weekofyear
    return row

def create_only_date_train_features(y_series):
    """
    Создает обучающий датасет из признаков, полученных из дат для y_series
    """

    time_features = pd.DataFrame(
        [create_date_features(date) for date in y_series.index]
    )
    return time_features, y_series


def create_date_and_shifted_train_features(
    y_series, shifts=5, week_seasonal_shifts=1, year_seasonal_shifts=1
):
    """
    Создает обучающий датасет из признаков, полученных из дат
    и значений ряда ранее.
    При этом используются значения ряда со сдвигами
    на неделю и год назад.

    Параметры:
        - y_series
            временной ряд.
        - shifts
            дневной сдвиг.
        - week_seasonal_shifts
            недельный сдвиг.
        - year_seasonal_shifts
            годовой сдвиг.
    """
    
    curr_df, y = create_only_date_train_features(y_series)
    curr_df.index = y_series.index

    # применяем сдвиг по дням
    for shift in range(1, shifts + 1):
        curr_df[f"shift_{shift}"] = y_series.shift(shift, axis=0)

    # применяем сдвиг по неделям
    for shift in range(1, week_seasonal_shifts + 1):
        curr_df[f"week_seasonal_shift_{shift}"] = y_series.shift(
            shift * 7, axis=0
        )

    # применяем сдвиг по годам
    for shift in range(1, year_seasonal_shifts + 1):
        curr_df[f"year_seasonal_shift_{shift}"] = y_series.shift(
            shift * 365, axis=0
        )
    y = y_series

    # удалим первые строчки с nan
    drop_indices = curr_df.index[curr_df.isna().sum(axis=1) > 0]
    curr_df = curr_df.drop(index=drop_indices)
    y = y.drop(index=drop_indices)
    return curr_df, y

In [ ]:
train.head(45)

date
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
2013-01-06    12
2013-01-07    10
2013-01-08     9
2013-01-09    12
2013-01-10     9
2013-01-11     9
2013-01-12     7
2013-01-13    10
2013-01-14    12
2013-01-15     5
2013-01-16     7
2013-01-17    16
2013-01-18     7
2013-01-19    18
2013-01-20    15
2013-01-21     8
2013-01-22     7
2013-01-23     9
2013-01-24     8
2013-01-25    14
2013-01-26    12
2013-01-27    12
2013-01-28    11
2013-01-29     6
2013-01-30     9
2013-01-31    13
2013-02-01    11
2013-02-02    21
2013-02-03    15
2013-02-04    14
2013-02-05     9
2013-02-06    10
2013-02-07    13
2013-02-08    11
2013-02-09    14
2013-02-10    11
2013-02-11    16
2013-02-12    11
2013-02-13    14
2013-02-14    10
Name: sales, dtype: int64

In [ ]:
helper = build_datasets(train, extract_hybrid_strategy_features, window_size=7, model_count=5)

In [ ]:
helper[0][0][:6], helper[0][1][:6]

(array([[13, 11, 14, 13, 10, 12, 10],
        [11, 14, 13, 10, 12, 10,  9],
        [14, 13, 10, 12, 10,  9, 12],
        [13, 10, 12, 10,  9, 12,  9],
        [10, 12, 10,  9, 12,  9,  9],
        [12, 10,  9, 12,  9,  9,  7]]),
 array([ 9, 12,  9,  9,  7, 10]))

In [ ]:
train.index[7:]

DatetimeIndex(['2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11',
               '2013-01-12', '2013-01-13', '2013-01-14', '2013-01-15',
               '2013-01-16', '2013-01-17',
               ...
               '2016-12-22', '2016-12-23', '2016-12-24', '2016-12-25',
               '2016-12-26', '2016-12-27', '2016-12-28', '2016-12-29',
               '2016-12-30', '2016-12-31'],
              dtype='datetime64[ns]', name='date', length=1454, freq=None)

In [ ]:
score_models(train, test, models = [LinearRegression.fit(helper[0][0], helper[0][1])])

TypeError: ignored